## `fastplotlib` demo

In [1]:
import numpy as np
import zmq
import fastplotlib as fpl

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

No windowing system present. Using surfaceless platform
No config found!
No config found!


Available devices:
✅ (default) | NVIDIA GeForce GTX 1080 Ti | DiscreteGPU | Vulkan | 555.42.06
❗ | llvmpipe (LLVM 15.0.7, 256 bits) | CPU | Vulkan | Mesa 23.2.1-1ubuntu3.1~22.04.2 (LLVM 15.0.7)
❗ | NVIDIA GeForce GTX 1080 Ti/PCIe/SSE2 | Unknown | OpenGL | 


### Check if GPU or rendering with CPU via lavapipe is available

In [2]:
if len(fpl.utils.gpu.enumerate_adapters()) < 1:
    raise IndexError("WGPU could not enumerate any adapters, fastplotlib will not work.")

### Setup zmq subscriber client

In [3]:
context = zmq.Context()
sub = context.socket(zmq.SUB)
sub.setsockopt(zmq.SUBSCRIBE, b"")

# keep only the most recent message
sub.setsockopt(zmq.CONFLATE, 1)

# address must match publisher in Processor actor
sub.connect("tcp://127.0.0.1:5555")

<SocketContext(connect='tcp://127.0.0.1:5555')>

In [4]:
def get_buffer():
    """
    Gets the buffer from the publisher
    """
    try:
        b = sub.recv(zmq.NOBLOCK)
    except zmq.Again:
        pass
    else:
        return b
    
    return None

In [5]:

# Create the figure
figure = fpl.Figure()

# Initialize sine and cosine wave data
xs = np.linspace(-10, 10, 100)
cos_ys = np.cos(xs)
sin_ys = np.sin(xs)

# Add a line plot placeholder
figure[0, 0].add_line(data=np.column_stack((xs, cos_ys)), name="wave")

def update_frame(p):
    # Receive memory with buffer
    buff = get_buffer()  # Replace with a valid data source
    
    if buff is not None:
        # Convert the buffer to a numpy array
        a = np.frombuffer(buff, dtype=np.float64)
        ix = a[-1]
        
        # Update wave plot based on parity of ix
        if ix % 2 == 0:
            # Display cosine wave
            new_ys = np.cos(xs)
            p["wave"].data = np.column_stack((xs, new_ys, np.zeros_like(xs)))
            p["wave"].color = "blue"  # Change color for cosine
        else:
            # Display sine wave
            new_ys = np.sin(xs)
            p["wave"].data = np.column_stack((xs, new_ys, np.zeros_like(xs)))
            p["wave"].color = "red"  # Change color for sine
        
        # Update the plot name to reflect the current frame
        p.name = f"frame: {int(ix)}"

# Add the animation update function
figure[0, 0].add_animations(update_frame)

# Show the figure
figure.show()


RFBOutputContext()

/home/edwin/edwin_improv/fpl/lib/python3.10/site-packages/fastplotlib/graphics/_features/_base.py:21: UserWarning: casting float64 array to float32
  warn(f"casting {array.dtype} array to float32")


JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

## `fastplotlib` is non-blocking!

In [6]:
figure.canvas.get_stats()

{'sent_frames': 123,
 'confirmed_frames': 121,
 'roundtrip': 0.05958017041860533,
 'delivery': -0.5480798748899097,
 'img_encoding': 0.007126040003315463,
 'fps': 31.430251931495025}

In [9]:
buff = get_buffer()

In [12]:
a= np.frombuffer(buff, dtype=np.float64)

In [13]:
a.shape

(262145,)

In [14]:
a[-1]

229.0

In [17]:
figure[0,0]["img"].data = a[:-1].reshape(512, 512)

In [25]:
figure[0,0].name = "bah"